In [12]:
%load_ext autoreload
%autoreload 2
import copy
from collections import OrderedDict, defaultdict

import numpy as np
import pandas as pd
from spikeextractors import RecordingExtractor, SortingExtractor
import spikeextractors as se

import spikemetrics.metrics as metrics
import spiketoolkit as st
from spikemetrics.utils import Epoch, printProgressBar
from spiketoolkit.curation.thresholdcurator import ThresholdCurator
import spiketoolkit as st

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
rec, sort = se.example_datasets.toy_example(duration=10, num_channels=4)

In [14]:
mc = st.validation.MetricCalculator(sort, rec)
mc.compute_all_metric_data(seed=0)

_ = mc.compute_metrics(seed=0)
metric_dict = mc.get_metrics_dict()
assert type(mc._recording._recording).__name__ == 'BandpassFilterRecording' #check if filter by default
assert 'firing_rate' in metric_dict.keys()
assert 'num_spikes' in metric_dict.keys()
assert 'isi_viol' in metric_dict.keys()
assert 'presence_ratio' in metric_dict.keys()
assert 'amplitude_cutoff' in metric_dict.keys()
assert 'max_drift' in metric_dict.keys()
assert 'cumulative_drift' in metric_dict.keys()
assert 'silhouette_score' in metric_dict.keys()
assert 'isolation_distance' in metric_dict.keys()
assert 'l_ratio' in metric_dict.keys()
assert 'd_prime' in metric_dict.keys()
assert 'nn_hit_rate' in metric_dict.keys()
assert 'nn_miss_rate' in metric_dict.keys()
assert 'snr' in metric_dict.keys()
assert mc.is_filtered()

In [15]:
from spiketoolkit.validation import AmplitudeCutoff
from spiketoolkit.validation import SilhouetteScore
from spiketoolkit.validation import MetricData
from spiketoolkit.validation.quality_metrics_new import compute_amplitude_cutoffs
from spiketoolkit.validation.quality_metrics_new import compute_silhouette_scores

from spiketoolkit.validation import NumSpikes
from spiketoolkit.validation.quality_metrics_new import compute_num_spikes

from spiketoolkit.validation import DPrime
from spiketoolkit.validation.quality_metrics_new import compute_d_primes

from spiketoolkit.validation import FiringRate
from spiketoolkit.validation.quality_metrics_new import compute_firing_rates

from spiketoolkit.validation import PresenceRatio
from spiketoolkit.validation.quality_metrics_new import compute_presence_ratios

from spiketoolkit.validation import LRatio
from spiketoolkit.validation.quality_metrics_new import compute_l_ratios

In [17]:
#num spikes
print("num spikes")
md = MetricData(sorting=sort, sampling_frequency=rec.get_sampling_frequency(), apply_filter=True)
ns = NumSpikes(metric_data=md)
ns_metric = compute_num_spikes(sorting=sort, sampling_frequency=rec.get_sampling_frequency())
print(np.equal(ns.compute_metric(), metric_dict['num_spikes']))
print(np.equal(ns_metric, metric_dict['num_spikes']))

#firing rate
print("firing rate")
md = MetricData(sorting=sort, sampling_frequency=rec.get_sampling_frequency(), apply_filter=True)
fr = FiringRate(metric_data=md)
fr_metric = compute_firing_rates(sorting=sort, sampling_frequency=rec.get_sampling_frequency())
print(np.equal(fr.compute_metric(), metric_dict['firing_rate']))
print(np.equal(fr_metric, metric_dict['firing_rate']))

#presence ratio
print("presence ratio")
md = MetricData(sorting=sort, sampling_frequency=rec.get_sampling_frequency(), apply_filter=True)
pr = PresenceRatio(metric_data=md)
pr_metric = compute_presence_ratios(sorting=sort, sampling_frequency=rec.get_sampling_frequency())
print(np.equal(pr.compute_metric(), metric_dict['presence_ratio']))
print(np.equal(pr_metric, metric_dict['presence_ratio']))

#amp cutoff
print("amp cutoff")
md = MetricData(sorting=sort, recording=rec, apply_filter=True)
md.compute_amplitudes(seed=0)
ac = AmplitudeCutoff(metric_data=md)
ac_metric = compute_amplitude_cutoffs(sorting=sort, recording=rec, seed=0)
print(np.equal(ac.compute_metric(), metric_dict['amplitude_cutoff']))
print(np.equal(ac_metric, metric_dict['amplitude_cutoff']))

#silhouette score
print("silhouette score")
md = MetricData(sorting=sort, recording=rec, apply_filter=True)
md.compute_pca_scores(seed=0)
ss = SilhouetteScore(metric_data=md)
ss_metric = compute_silhouette_scores(sorting=sort, recording=rec, seed=0)
print(np.equal(ss.compute_metric(seed=0), metric_dict['silhouette_score']))
print(np.equal(ss_metric, metric_dict['silhouette_score']))

#d primes
print("d primes")
md = MetricData(sorting=sort, recording=rec, apply_filter=True)
md.compute_pca_scores(seed=0)
dp = DPrime(metric_data=md)
dp_metric = compute_d_primes(sorting=sort, recording=rec, seed=0)
print(np.equal(dp.compute_metric(seed=0), metric_dict['d_prime']))
print(np.equal(dp_metric, metric_dict['d_prime']))

#d primes
print("l ratios")
md = MetricData(sorting=sort, recording=rec, apply_filter=True)
md.compute_pca_scores(seed=0)
lr = LRatio(metric_data=md)
lr_metric = compute_l_ratios(sorting=sort, recording=rec, seed=0)
print(np.equal(lr.compute_metric(seed=0), metric_dict['l_ratio']))
print(np.equal(lr_metric, metric_dict['l_ratio']))

num spikes
[[ True  True  True  True  True  True  True  True  True  True]]
[[ True  True  True  True  True  True  True  True  True  True]]
firing rate
[[ True  True  True  True  True  True  True  True  True  True]]
[[ True  True  True  True  True  True  True  True  True  True]]
presence ratio
[[ True  True  True  True  True  True  True  True  True  True]]
[[ True  True  True  True  True  True  True  True  True  True]]
amp cutoff
[[ True  True  True  True  True  True  True  True  True  True]]
[[ True  True  True  True  True  True  True  True  True  True]]
silhouette score
[[ True  True  True  True  True  True  True  True  True  True]]
[[ True  True  True  True  True  True  True  True  True  True]]
d primes
[[ True  True  True  True  True  True  True  True  True  True]]
[[ True  True  True  True  True  True  True  True  True  True]]
l ratios
[[ True  True  True  True  True  True  True  True  True  True]]
[[ True  True  True  True  True  True  True  True  True  True]]


In [19]:
from spiketoolkit.curation.threshold_metrics import threshold_amplitude_cutoffs
from spiketoolkit.curation.threshold_metrics import threshold_silhouette_scores
from spiketoolkit.curation.threshold_metrics import threshold_num_spikes
from spiketoolkit.curation.threshold_metrics import threshold_d_primes
from spiketoolkit.curation.threshold_metrics import threshold_firing_rates
from spiketoolkit.curation.threshold_metrics import threshold_presence_ratios
from spiketoolkit.curation.threshold_metrics import threshold_l_ratios

In [21]:
print("amp cutoff")
sorting_new = threshold_amplitude_cutoffs(sorting=sort, recording=rec, threshold=.4, threshold_sign='greater')
sorting_old = st.curation.threshold_amplitude_cutoffs(sorting=sort, recording=rec, threshold=.4, threshold_sign='greater')
print(compute_amplitude_cutoffs(sorting=sorting_new, recording=rec))
print(np.equal(compute_amplitude_cutoffs(sorting=sorting_new, recording=rec), compute_amplitude_cutoffs(sorting=sorting_old, recording=rec)))

print("silhouette scores")
sorting_new = threshold_silhouette_scores(sorting=sort, recording=rec, threshold=.4, threshold_sign='greater', seed=0)
sorting_old = st.curation.threshold_silhouette_scores(sorting=sort, recording=rec, threshold=.4, threshold_sign='greater', seed=0)
print(compute_silhouette_scores(sorting=sorting_new, recording=rec, seed=0))
print(np.equal(compute_silhouette_scores(sorting=sorting_new, recording=rec, seed=0), compute_silhouette_scores(sorting=sorting_old, recording=rec, seed=0)))

print("num spikes")
sorting_new = threshold_num_spikes(sorting=sort, threshold=24, threshold_sign='less')
sorting_old = st.curation.threshold_num_spikes(sorting=sort, threshold=24, threshold_sign='less')
print(compute_num_spikes(sorting=sorting_new))
print(np.equal(compute_num_spikes(sorting=sorting_new), compute_num_spikes(sorting=sorting_old)))

print("firing rate")
sorting_new = threshold_firing_rates(sorting=sort, threshold=2.5, threshold_sign='less')
sorting_old = st.curation.threshold_firing_rates(sorting=sort, threshold=2.5, threshold_sign='less')
print(compute_firing_rates(sorting=sorting_new))
print(np.equal(compute_firing_rates(sorting=sorting_new), compute_firing_rates(sorting=sorting_old)))

print("presence ratio")
sorting_new = threshold_presence_ratios(sorting=sort, threshold=.19, threshold_sign='less_or_equal')
sorting_old = st.curation.threshold_presence_ratios(sorting=sort, threshold=.19, threshold_sign='less_or_equal')
print(compute_presence_ratios(sorting=sorting_new))
print(np.equal(compute_presence_ratios(sorting=sorting_new), compute_presence_ratios(sorting=sorting_old)))

print("d primes")
sorting_new = threshold_d_primes(sorting=sort, recording=rec, threshold=5, threshold_sign='greater', seed=0)
sorting_old = st.curation.threshold_d_primes(sorting=sort, recording=rec, threshold=5, threshold_sign='greater', seed=0)
print(compute_d_primes(sorting=sorting_new, recording=rec, seed=0))
print(np.equal(compute_d_primes(sorting=sorting_new, recording=rec, seed=0), compute_d_primes(sorting=sorting_old, recording=rec, seed=0)))

print("l ratios")
sorting_new = threshold_l_ratios(sorting=sort, recording=rec, threshold=0, threshold_sign='greater', seed=0)
sorting_old = st.curation.threshold_l_ratios(sorting=sort, recording=rec, threshold=0, threshold_sign='greater', seed=0)
print(compute_l_ratios(sorting=sorting_new, recording=rec, seed=0))
print(np.equal(compute_l_ratios(sorting=sorting_new, recording=rec, seed=0), compute_l_ratios(sorting=sorting_old, recording=rec, seed=0)))

amp cutoff
[array([0.01176284, 0.0118967 , 0.01078261, 0.00958454])]
[[ True  True  True  True]]
silhouette scores
[array([0.14981897, 0.14981897, 0.16580476])]
[[ True  True  True]]
num spikes
[array([24., 24., 26., 27., 24.])]
[[ True  True  True  True  True]]
firing rate
[array([2.80662363, 2.91457069])]
[[ True  True]]
presence ratio
[array([0.19, 0.22, 0.2 , 0.18, 0.21])]
[[ True  True  True  True  True]]
d primes
[array([5.45161891, 9.26523897, 4.678117  , 7.32783486, 7.29554003])]
[[ True  True  True  True  True]]
l ratios
[array([0., 0., 0., 0., 0., 0., 0.])]
[[ True  True  True  True  True  True  True]]
